In [ ]:
# first run installations and data download
#! pip install -r requirements.txt
#! mkdir -p './data/'

In [2]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata

from pymilvus import connections
from llama_index.vector_stores.milvus import MilvusVectorStore

In [ ]:
#default_server.stop()
#default_server.cleanup()

In [ ]:
from dotenv import load_dotenv
import os
import openai
load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
from milvus import default_server
default_server.start()

In [ ]:
connections.connect(host='127.0.0.1', port=default_server.listen_port)

In [3]:
# try:
#     storage_context = StorageContext.from_defaults(
#         persist_dir="./storage/pep"
#     )
#     pep_index = load_index_from_storage(storage_context)

#     index_loaded = True
# except:
    # index_loaded = False
index_loaded=False

In [6]:
if not index_loaded:
    # load data
    docs = SimpleDirectoryReader(
        input_dir="./data/"
    ).load_data()

    # build index
    vector_store_pep = MilvusVectorStore(dim=1536, collection_name="pep", overwrite=True) 
    storage_context_pep = StorageContext.from_defaults(vector_store=vector_store_pep) 
    pep_index = VectorStoreIndex.from_documents(docs, storage_context=storage_context_pep)

    # persist index
    pep_index.storage_context.persist(persist_dir="./storage/pep") 

In [ ]:
pep_engine = pep_index.as_query_engine(similarity_top_k=3)

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=pep_engine,
        metadata=ToolMetadata(
            name="pep",
            description=(
                "Provides information about python styling. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    )
]

In [ ]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI

/Users/yujiantang/Documents/workspace/ai_agents_cookbooks/newenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


llm = OpenAI(model="gpt-3.5-turbo-0613")

agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    # context=context
)

In [ ]:
response = agent.chat("What are the coding conventions for the C code comprising the C implementation of Python?")
print(str(response))